Notebook to take into account the relative abundances of species when calculating the average number of gene copies across species of a same genus / family.
- data too large to be included in this Git repository 
- data available on demand: albane.ruaud@tuebingen.mpg.de

# Preambule

In [1]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(qs)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


qs v0.23.4.



# Taxa

In [2]:
# taxa used in the model
feats <- qread('../data/taxa_not_flt.qs') %>%
    select(-Sample)
feats <- data.frame(Feature = colnames(feats), Abundance_avg = colMeans(feats))

In [3]:
feats <- feats %>% subset(Abundance_avg > 0 ) 
feats %>% dim

[1] 4733    2

In [4]:
feats %>% head(2)

,Feature,Abundance_avg
,<chr>,<dbl>
s_1_14_0_10_34_76_sp002763075,s_1_14_0_10_34_76_sp002763075,6.173400e-07
s_1_19_sp001922585,s_1_19_sp001922585,4.584657e-07


# llcds results

In [5]:
# hyddb
hyddb <- fread('/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/hydDB/hyddb-results.csv'
               , sep=',', header=FALSE) %>%
    select(-V6, -V7, -V8) 
hyddb$V3 <- hyddb$V3 %>% str_replace(pattern = ' +$', replacement = '')

In [6]:
# re-loading table
hits <- fread('/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/llcds_hyd/annotate/dmnd_hits_all.tsv.gz')

In [7]:
# filtering
hits_f <- hits %>% subset(pident >= 50 & length / qlen > 0.8)

In [8]:
hits %>% dim
hits_f %>% dim

[1] 3737934      14

[1] 111187     14

In [9]:
hits_f <- hits_f %>% left_join(hyddb, c('sseqid'='V3'))

In [10]:
hits_f %>% dim

[1] 111300     25

# Add taxonomic level

In [11]:
aa_files_f <- read.table('/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/samples.tsv'
                        , sep = '\t', header = TRUE)  %>%
    select(Sample, Phylum, Class, Order, Family, Genus, Species) %>%
    rename('Genome' = Sample)

In [12]:
# all taxa lacking hits
hits_f <- aa_files_f %>%
    left_join(hits_f, by = 'Genome')
hits_f$V5[is.na(hits_f$V5)] <- 'None'

In [13]:
hits_f %>% colnames

[1] "Genome"   "Phylum"   "Class"    "Order"    "Family"   "Genus"   
 [7] "Species"  "qseqid"   "sseqid"   "pident"   "length"   "mismatch"
[13] "gapopen"  "qstart"   "qend"     "sstart"   "send"     "evalue"  
[19] "bitscore" "qlen"     "V1"       "V2"       "V4"       "V5"      
[25] "V9"       "V10"      "V11"      "V12"      "V13"      "V14"     
[31] "V15"

In [14]:
hits_tax <- hits_f %>% select(Genome, Phylum, Class, Order, Family, Genus, Species, V5, pident) %>% 
                subset(Species %in% feats$Feature)

# Weight by genome size

In [15]:
samps_tbl_file <- read.table(header = TRUE, sep = '\t', dec = '.', fill = TRUE
 , '/ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/Bacteria_marker_genes/data/samples.tsv')

In [16]:
struo_meta <- fread('/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/metadata_1per-GTDB-Spec_gte50comp-lt5cont_wtaxID_wPath.tsv')

In [17]:
struo_meta$Genome <- str_replace(struo_meta$gtdb_genome_representative, pattern = '\\.', replacement = '_')

In [18]:
hits_tax$Genome_short <- str_extract(hits_tax$Genome, pattern = '^[:upper:]+\\_[:upper:]+\\_[:digit:]+\\_[:digit:]+')

In [19]:
sum(hits_tax$Genome_short %in% struo_meta$Genome)
length(hits_tax$Genome)

[1] 53703

[1] 53703

In [20]:
hits_tax <- left_join(hits_tax,  select(struo_meta, Genome, genome_size), by = c('Genome_short' = 'Genome'))

In [21]:
dim(hits_tax)

[1] 53703    11

# Per taxonomic level

## species

In [22]:
pp <- hits_tax %>% group_by(Phylum, Class, Order, Family, Genus, Species, V5) %>% 
        summarise(n_copies = n(), pident = mean(pident, na.rm = TRUE), genome_size = unique(genome_size))

`summarise()` regrouping output by 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species' (override with `.groups` argument)



In [23]:
pp$r_copies <- pp$n_copies/pp$genome_size
pp <- select(pp, -n_copies, -genome_size)

## add all missing categories for each species

In [24]:
sp <- pp$Species %>% unique
hctg <- pp$V5 %>% unique

In [25]:
# create all combinations of species and hydrogenases needed
to_add <- expand.grid(sp, hctg) %>% as.data.frame
colnames(to_add) <- c('Species', 'V5')
    
# keep only the ones missing: give them a 0 count number and NA pident
to_add <- setdiff(to_add, pp %>% ungroup %>% select(Species, V5)) %>% 
                # add the taxonomic levels
                left_join(pp%>% ungroup %>% select(Phylum, Class, Order,Family, Genus, Species), by = 'Species') 
to_add$r_copies <- 0
to_add$pident <- NaN
    
# add them to the dataset
pp <- rbind(pp, to_add)

In [26]:
pp %>% tail(2)

Phylum,Class,Order,Family,Genus,Species,V5,pident,r_copies
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,g_Akkermansia,s_Akkermansia_sp001580195,[NiFe] Group 3c,NaN,0
p_Verrucomicrobiota,c_Verrucomicrobiae,o_Verrucomicrobiales,f_Akkermansiaceae,g_Akkermansia,s_Akkermansia_sp002358905,[NiFe] Group 3c,NaN,0


## genera

In [27]:
gen <- pp %>% left_join(feats, by = c('Species' = 'Feature')) %>% 
        group_by(Phylum, Class, Order, Family, Genus, V5) %>% 
        summarise(r_copies = weighted.mean(x = r_copies, w = Abundance_avg, na.rm = TRUE)
                  , pident =  weighted.mean(x = pident, w = Abundance_avg, na.rm = TRUE))
gen$Feature <- gen$Genus 

`summarise()` regrouping output by 'Phylum', 'Class', 'Order', 'Family', 'Genus' (override with `.groups` argument)



## family

In [28]:
fam <- pp %>% left_join(feats, by = c('Species' = 'Feature')) %>% 
        group_by(Phylum, Class, Order, Family, V5) %>% 
        summarise(r_copies = weighted.mean(x = r_copies, w = Abundance_avg, na.rm = TRUE)
                  , pident =  weighted.mean(x = pident, w = Abundance_avg, na.rm = TRUE))
fam$Feature <- fam$Family

`summarise()` regrouping output by 'Phylum', 'Class', 'Order', 'Family' (override with `.groups` argument)



## merge

In [29]:
colnames(pp)
colnames(gen)
colnames(fam)

[1] "Phylum"   "Class"    "Order"    "Family"   "Genus"    "Species"  "V5"      
[8] "pident"   "r_copies"

[1] "Phylum"   "Class"    "Order"    "Family"   "Genus"    "V5"       "r_copies"
[8] "pident"   "Feature"

[1] "Phylum"   "Class"    "Order"    "Family"   "V5"       "r_copies" "pident"  
[8] "Feature"

In [30]:
gen$Species <- NA
fam$Genus <- NA
fam$Species <- NA
pp$Feature <- pp$Species

In [31]:
nrow(pp)
nrow(gen)
nrow(fam)

[1] 181348

[1] 30112

[1] 5856

In [32]:
pp <- union(pp, gen) %>% union(fam)

In [33]:
nrow(pp) #130080

[1] 130080

In [35]:
(!(unique(pp$Feature) %in% feats$Feature)) %>% sum

[1] 0

In [36]:
#write.table(pp, file = '../data/llcds_hyd/hyd-annot_byPredFeat_byHyd_weightedRA.txt'
#            , sep = '\t', quote = FALSE)

# Formatting

In [37]:
colnames(pp)[colnames(pp) == 'V5'] <- 'hyd_ctg'

In [38]:
feat_imp <- qread('../tmp/Variable_importance.qs')
hyd_pergene <- pp %>% ungroup %>% select(-Phylum, -Class, -Order, -Family, -Genus, -Species) %>% 
                    left_join(feat_imp, by = 'Feature')

## add descriptions

In [39]:
hyd_description <- read.table('hydDB_curated.txt', sep = '\t', header = TRUE)

In [40]:
hyd_description %>% subset(pred_function %in% c('Bifurcating', 'Bidirectional'))

,hyd_ctg,pred_function,electron_donor_acceptor,O2,comment
,<chr>,<chr>,<chr>,<chr>,<chr>
1,[Fe],Bidirectional,Hmd,tolerant,Reduction of Hmd during hydrogenotrophic methanogenesis
4,[FeFe] Group A3,Bifurcating,"NAD, ferredoxin",labile,
26,[NiFe] Group 3a,Bidirectional,F420,tolerant,Reduction of F420 during hydrogenotrophic methanogenesis
27,[NiFe] Group 3b,Bidirectional,NADP,tolerant,Directly couples oxidation of NADPH to fermentative evolution of H2. Diverse soil and aquatic environments.
28,[NiFe] Group 3c,Bifurcating,heterodisulfide and ferredoxin,tolerant,In methanogens
29,[NiFe] Group 3d,Bidirectional,NAD,tolerant,


In [41]:
hyd_pergene <- hyd_pergene %>% left_join(hyd_description %>% select(-comment), by = 'hyd_ctg')

In [42]:
hyd_pergene <- hyd_pergene %>% subset(!(str_detect(Feature, pattern = '^(f|g)\\_') & hyd_ctg == 'None') )

In [43]:
# simplify electron donor/acceptor variable: Sulfate and others if sulfate listed
hyd_pergene$electron_donor_acceptor <- ifelse(str_detect(hyd_pergene$electron_donor_acceptor, pattern = 'Sulfate')
                                             , 'Sulfate and others', hyd_pergene$electron_donor_acceptor)
# simplify: keep only the interesting labels (in tmp) and group the others
tmp <- c('Ferredoxin', 'NAD', 'NAD, ferredoxin', 'NADP', 'Formate', 'Sulfate and others')
hyd_pergene$electron_donor_acceptor <- ifelse(!(hyd_pergene$electron_donor_acceptor %in% tmp), 'Other', hyd_pergene$electron_donor_acceptor)

In [44]:
hyd_pergene$pred_general <- ifelse(hyd_pergene$pred_function %in% c('Bifurcating', 'Evolving'), 'H2 producing'
                                   , hyd_pergene$pred_function)

In [45]:
# this one is bifurcating but actually involved in H2 uptake
hyd_pergene$pred_general[ hyd_pergene$hyd_ctg == '[NiFe] Group 3c' ] <- 'H2 uptake'

In [46]:
tmpFun <- function(x){ifelse(is.na(x), 0, x)}
hyd_pergene <- hyd_pergene %>% mutate_at(vars(Gini, importance), tmpFun)

In [47]:
colnames(hyd_pergene) <- str_replace(colnames(hyd_pergene), pattern = 'importance', replacement = 'endoR')

In [48]:
qsave(hyd_pergene, 'HydDB_res_r.qs')